# Limpieza y Análisis de Datos - Semana 3

In [ ]:
# Importamos las librerias necesarias 
import pandas as pd
import matplotlib.pyplot as plt

## --- Viajes ---
#### Preguntas h, j, t

In [ ]:
def viajes():
    print("Viajes")

## --- Ciudades ---
#### Preguntas o-s, f

In [ ]:
def ciudades():
    print("Ciudades")

## --- Tiendas ---
#### Preguntas l - s

In [ ]:
def tiendas():
    print("Tiendas")

## --- Vehiculos --- 
#### Preguntas a - e, g, i, k

In [ ]:
def vehiculos():
    print("Vehiculos")

In [ ]:
def limpiezaDeDatos(df):
    # Quitar columnas sin nombre
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
def merge(df_1, df_2):
    print("Combinando archivos para eliminar repetidos...")


In [ ]:
def main():
    print("Main, aqui llamamos a las demas funciones")